[How to Read Outlook Emails by Python](#https://www.codementor.io/aliacetrefli/how-to-read-outlook-emails-by-python-jkp2ksk95)

In [ ]:
import win32com.client
import win32com
import os
import sys

In [ ]:
f = open("testfile.txt","w+")

In [ ]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
accounts= win32com.client.Dispatch("Outlook.Application").Session.Accounts;

In [ ]:
def Apple(folder):
    messages = folder.Items
    a=len(messages)
    if a>0:
        for message2 in messages:
            try:
                sender = message2.SenderEmailAddress
                if sender != "":
                    print(sender, file=f)
            except:
                print("Error")
                print(account.DeliveryStore.DisplayName)
                pass
            try:
                message2.Save
                message2.Close(0)
            except:
                 pass

In [ ]:
for account in accounts:
    global inbox
    inbox = outlook.Folders(account.DeliveryStore.DisplayName)
    print("****Account Name**********************************",file=f)
    print(account.DisplayName,file=f)
    print(account.DisplayName)
    print("***************************************************",file=f)
    folders = inbox.Folders

    for folder in folders:
        print("****Folder Name**********************************", file=f)
        print(folder, file=f)
        print("*************************************************", file=f)
        Apple(folder)
        a = len(folder.folders)

        if a>0 :
            global z
            z = outlook.Folders(account.DeliveryStore.DisplayName).Folders(folder.name)
            x = z.Folders
            for y in x:
                Apple(y)
                print("****Folder Name**********************************", file=f)
                print("..."+y.name,file=f)
                print("*************************************************", file=f)
print("Finished Succesfully")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('testfile.txt', sep='\n', header=None)

In [ ]:
df.head()

In [ ]:
df = df[df[0].str.contains('@')]

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df = df[0].str.lower()

In [ ]:
df.value_counts()

* [Reading e-mails from Outlook with Python through MAPI](#https://stackoverflow.com/questions/5077625/reading-e-mails-from-outlook-with-python-through-mapi)
* [MailItem Interface](#https://docs.microsoft.com/en-us/dotnet/api/microsoft.office.interop.outlook.mailitem?redirectedfrom=MSDN&view=outlook-pia#properties_)
* [OlDefaultFolders Enumeration (Outlook)](#https://docs.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders)
* [Items Object (Outlook)](#https://docs.microsoft.com/en-us/office/vba/api/outlook.items)
* [ContactItem Object (Outlook)](#https://docs.microsoft.com/en-us/office/vba/api/outlook.contactitem)

In [ ]:
inbox = outlook.GetDefaultFolder(6)

In [ ]:
inbox.Folders

In [ ]:
messages = inbox.Items

In [ ]:
messages(7000).Display()  # opens the email

In [ ]:
message = messages.GetFirst()

In [ ]:
print(message.Subject)
print(message.To)
print(message.ReceivedTime)
print(message.Sender)
print(message.SenderEmailAddress)
print(message.SentOn)

In [ ]:
print(message.body)

In [ ]:
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items
y = messages.GetLast()
print(f'Type: {type(y.body)}')
print(y.body)
print('*' * 100)
print('*' * 100)
i = 0
while messages:
    i+=1
    y = messages.GetPrevious()
    print(f'Type: {type(y.body)}')
    print(y.body)
    print('*' * 100)
    print('*' * 100)
#     print('\n')
    if i > 10:
        break